In [14]:
import pandas as pd
import numpy as np
import glob
import os
import re

In [73]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\franc\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [15]:
df = pd.read_csv('thread_df/thread_df_1_9.csv', index_col=0)
comments = df.dropna(subset=['comment']).reset_index(drop = True)

In [16]:
comments['comment'].tolist()

['Thank you for participating, everyone! Unfortunately, we weren’t quite able to close the gap on this SL550 but we will be in contact with the high bidder, @edstutzm, and the seller to make a deal!',
 'Reserve not met, bid to $15,277',
 'Re: Nomodsplease\nYes, the multi contour seats work with these features.',
 'Do the multi-contour seats work? In other words - the air bladders for lumbar, bolsters, massage etc?',
 'Re: salehi\nHere is the driving link to view,\nhttps://youtu.be/CnxbE9DXSSM',
 'Re: salehi\nHello. The cold start video has been posted. I can work on posting a video of driving as well.',
 'Would you be able to post a video that shows a cold start and another one where you are driving the car please?',
 'I love seeing these when they’ve been loved and enjoyed as opposed to parked in a garage and never driven. That is so rad. It’ll be a good deal for someone to enjoy for another 100k that isn’t hung up on cosmetics.',
 "Hi All, it's Vaughn here to answer any questions tha

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
#sentiment_pipeline = pipeline("sentiment-analysis")
#model = pipeline(model="nlptown/bert-base-multilingual-uncased-sentiment")
model = pipeline(model="cardiffnlp/twitter-roberta-base-sentiment-latest")
ds = model(comments['comment'].tolist())

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
ds

[{'label': 'neutral', 'score': 0.4099806845188141},
 {'label': 'negative', 'score': 0.5645845532417297},
 {'label': 'neutral', 'score': 0.677177369594574},
 {'label': 'neutral', 'score': 0.8985506296157837},
 {'label': 'neutral', 'score': 0.8650327324867249},
 {'label': 'neutral', 'score': 0.705142080783844},
 {'label': 'neutral', 'score': 0.9347090721130371},
 {'label': 'positive', 'score': 0.9725500345230103},
 {'label': 'positive', 'score': 0.7882378697395325}]

In [9]:
result = pd.DataFrame(ds)['label'].apply(lambda x: 1 if x == 'positive' else (0 if x == 'neutral' else -1))

In [17]:
#result = pd.DataFrame(ds)['label'].str.extract('(\d+)').astype(int)

comments = comments.join(result).rename(columns={'label':'sentiment_score'})

In [18]:
comments

,position,user,comment,bid,upvote,rep,seller,bidder,verified,sentiment_score
0,1,Cars&Bids,"Thank you for participating, everyone! Unfortu...",NaN,0\nReply,Reputation Icon32k,0,0,0,0
1,2,NaN,"Reserve not met, bid to $15,277",NaN,NaN,NaN,0,0,0,-1
2,4,bitransit,"Re: Nomodsplease\nYes, the multi contour seats...",NaN,4\nReply\nFlag as inappropriate,Reputation Icon17,1,0,0,0
3,5,Nomodsplease,Do the multi-contour seats work? In other word...,NaN,1\nReply\nFlag as inappropriate,Reputation Icon10,0,0,0,0
4,7,bitransit,"Re: salehi\nHere is the driving link to view,\...",NaN,3\nReply\nFlag as inappropriate,Reputation Icon17,1,0,0,0
5,10,bitransit,Re: salehi\nHello. The cold start video has be...,NaN,3\nReply\nFlag as inappropriate,Reputation Icon17,1,0,0,0
6,11,salehi,Would you be able to post a video that shows a...,NaN,2\nReply\nFlag as inappropriate,Reputation Icon31,0,1,1,0
7,15,my3swedes,I love seeing these when they’ve been loved an...,NaN,1\nReply\nFlag as inappropriate,Reputation Icon535,0,0,1,1
8,17,bitransit,"Hi All, it's Vaughn here to answer any questio...",NaN,1\nReply\nFlag as inappropriate,Reputation Icon17,1,0,0,1


In [19]:
comments['sentiment_score'].sum()/len(comments)

0.1111111111111111

In [17]:
sia = SentimentIntensityAnalyzer()
sia.polarity_scores(text)

{'neg': 0.0, 'neu': 0.66, 'pos': 0.34, 'compound': 0.9552}

In [36]:
sia = SentimentIntensityAnalyzer()
def get_sentiment_score(text):
    return sia.polarity_scores(text)['compound']

In [17]:
comments['sentiment_score'] = comments['comment'].apply(get_sentiment_score)

C:\Users\franc\AppData\Local\Temp\ipykernel_25176\3215188887.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['sentiment_score'] = comments['comment'].apply(get_sentiment_score)


In [3]:
import re
def mark_relevant(dfi):
    df = dfi.copy()
    df['sold_msg'] = df.comment.str.match(pat = '^Sold to .*')
    if df['sold_msg'].astype(int).sum() == 0:
        df['sold_msg'] = df.comment.str.match(pat = '^Reserve not met.*')
    sold_pos = df.loc[df['sold_msg'] == True].iloc[0,0]
    df['relevant'] = df['position'] >= sold_pos
    df.drop(columns = ['sold_msg'], inplace = True)
    return df

In [4]:
test = mark_relevant(comments)
relevant_comments = test[test['relevant'] == True].iloc[1:] # get rid of irrelevant comments after car selling
final_sentiment = relevant_comments.sentiment_score.sum()/len(relevant_comments)

NameError: name 'comments' is not defined

In [33]:
relevant_comments

,position,user,comment,bid,upvote,rep,seller,bidder,verified,sentiment_score,relevant
3,4,bitransit,"Re: Nomodsplease\nYes, the multi contour seats...",NaN,4\nReply\nFlag as inappropriate,Reputation Icon17,1,0,0,0.4019,True
4,5,Nomodsplease,Do the multi-contour seats work? In other word...,NaN,1\nReply\nFlag as inappropriate,Reputation Icon10,0,0,0,0.0000,True
6,7,bitransit,"Re: salehi\nHere is the driving link to view,\...",NaN,3\nReply\nFlag as inappropriate,Reputation Icon17,1,0,0,0.0000,True
9,10,bitransit,Re: salehi\nHello. The cold start video has be...,NaN,3\nReply\nFlag as inappropriate,Reputation Icon17,1,0,0,0.2732,True
10,11,salehi,Would you be able to post a video that shows a...,NaN,2\nReply\nFlag as inappropriate,Reputation Icon31,0,1,1,0.3182,True
14,15,my3swedes,I love seeing these when they’ve been loved an...,NaN,1\nReply\nFlag as inappropriate,Reputation Icon535,0,0,1,0.9552,True
16,17,bitransit,"Hi All, it's Vaughn here to answer any questio...",NaN,1\nReply\nFlag as inappropriate,Reputation Icon17,1,0,0,0.6114,True


In [69]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
sentiment_pipeline = pipeline("sentiment-analysis", truncation = True)
model = pipeline(model="nlptown/bert-base-multilingual-uncased-sentiment", truncation = True)
#model = pipeline(model="cardiffnlp/twitter-roberta-base-sentiment-latest")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [70]:
all_files = glob.glob(os.path.join('thread_df', '*.csv'))

In [71]:
df_out = pd.DataFrame(columns = ['id', 'sentiment_score'])

for file in all_files:
    df = pd.read_csv(file, index_col=0)
    id = file.replace('thread_df\\thread_df_', '').replace('.csv','')
    comments = df.dropna(subset = ['comment']).reset_index(drop=True)
    ds = model(comments['comment'].tolist(), truncation = True)
    #comments.loc[:,'sentiment_score'] = comments['comment'].apply(get_sentiment_score)
    scores = pd.DataFrame(ds)['label'].str.extract('(\d+)').astype(int)
    comments = comments.join(scores).rename(columns={0:'sentiment_score'})
    test = mark_relevant(comments)
    relevant_comments = test[test['relevant'] == True].iloc[1:]
    final_sentiment = relevant_comments.sentiment_score.sum()/len(relevant_comments)
    df_out.loc[len(df_out.index)] = [id, final_sentiment]

KeyboardInterrupt: 

In [ ]:
df_out.to_csv('thread_sentiment.csv')